# Lab Exam - Set 4

This notebook contains implementations for all questions in Set 4.

## Question 13: Outlier Detection and Treatment using IQR Method

**Concepts:**
- **Outliers**: Data points that are significantly different from other observations
- **IQR (Interquartile Range)**: Q3 - Q1, measures statistical dispersion
- **Q1 (First Quartile)**: 25th percentile of data
- **Q3 (Third Quartile)**: 75th percentile of data
- **Outlier bounds**: Values outside [Q1 - 1.5×IQR, Q3 + 1.5×IQR] are outliers
- **Median replacement**: Replace extreme values with median (robust to outliers)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Create a sample dataset with outliers
np.random.seed(42)
data = {
    'Student_ID': range(1, 51),
    'Math_Score': np.random.normal(75, 10, 50),
    'Science_Score': np.random.normal(80, 12, 50),
    'English_Score': np.random.normal(70, 8, 50),
    'Study_Hours': np.random.normal(5, 1.5, 50)
}

# Add some intentional outliers
data['Math_Score'][5] = 150  # Extreme outlier
data['Math_Score'][10] = 10  # Extreme outlier
data['Science_Score'][15] = 200  # Extreme outlier
data['Study_Hours'][20] = 15  # Outlier

df = pd.DataFrame(data)
print("Original Dataset:")
print(df.head(10))
print("\nDataset Statistics:")
print(df.describe())

# Function to detect outliers using IQR method
def detect_outliers_iqr(data, column):
    """
    Detect outliers in a column using IQR method
    Returns: outliers dataframe, lower bound, upper bound
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # Calculate bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Find outliers
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    return outliers, lower_bound, upper_bound

# Function to replace outliers with median
def replace_outliers_with_median(data, column):
    """
    Replace outliers in a column with the median value
    Returns: cleaned column
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Calculate median
    median_value = data[column].median()
    
    # Replace outliers with median
    cleaned_column = data[column].copy()
    cleaned_column[(cleaned_column < lower_bound) | (cleaned_column > upper_bound)] = median_value
    
    return cleaned_column

# Detect outliers in all numeric columns
numeric_columns = ['Math_Score', 'Science_Score', 'English_Score', 'Study_Hours']

print("\n" + "="*80)
print("OUTLIER DETECTION RESULTS")
print("="*80)

for col in numeric_columns:
    outliers, lower, upper = detect_outliers_iqr(df, col)
    print(f"\n{col}:")
    print(f"  Lower Bound: {lower:.2f}")
    print(f"  Upper Bound: {upper:.2f}")
    
    if not outliers.empty:
        print(f"  Outliers Found: {len(outliers)}")
        print(f"  Outlier Values: {outliers[col].values}")
    else:
        print("  No outliers found")

# Create a copy of dataframe for cleaning
df_cleaned = df.copy()

# Replace outliers with median for all numeric columns
for col in numeric_columns:
    df_cleaned[col] = replace_outliers_with_median(df, col)

print("\n" + "="*80)
print("AFTER REPLACING OUTLIERS WITH MEDIAN")
print("="*80)
print(df_cleaned.describe())

# Visualize before and after comparison
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle('Outlier Detection and Treatment using IQR Method', fontsize=16, fontweight='bold')

for idx, col in enumerate(numeric_columns):
    # Before - Box plot
    axes[0, idx].boxplot(df[col], vert=True)
    axes[0, idx].set_title(f'{col}\n(Before Treatment)', fontweight='bold')
    axes[0, idx].set_ylabel('Value')
    axes[0, idx].grid(True, alpha=0.3)
    
    # After - Box plot
    axes[1, idx].boxplot(df_cleaned[col], vert=True)
    axes[1, idx].set_title(f'{col}\n(After Treatment)', fontweight='bold')
    axes[1, idx].set_ylabel('Value')
    axes[1, idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Summary statistics comparison
print("\n" + "="*80)
print("COMPARISON OF STATISTICS (Before vs After)")
print("="*80)
for col in numeric_columns:
    print(f"\n{col}:")
    print(f"  Mean: {df[col].mean():.2f} → {df_cleaned[col].mean():.2f}")
    print(f"  Std Dev: {df[col].std():.2f} → {df_cleaned[col].std():.2f}")
    print(f"  Min: {df[col].min():.2f} → {df_cleaned[col].min():.2f}")
    print(f"  Max: {df[col].max():.2f} → {df_cleaned[col].max():.2f}")

## Question 14: Correlation and Feature Importance Analysis

**Concepts:**
- **Correlation**: Measure of relationship between two variables (-1 to +1)
- **Positive correlation**: Both variables increase together
- **Negative correlation**: One increases while other decreases
- **Pearson correlation**: Measures linear relationship between variables
- **Feature importance**: Identifies which features most influence the target
- **Heatmap**: Visual representation of correlation matrix
- **Random Forest**: Tree-based model that provides feature importance scores

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from scipy.stats import pearsonr

# Create a comprehensive dataset
np.random.seed(42)
n_samples = 200

data = {
    'Study_Hours': np.random.uniform(1, 10, n_samples),
    'Previous_Score': np.random.uniform(50, 95, n_samples),
    'Sleep_Hours': np.random.uniform(4, 9, n_samples),
    'Attendance': np.random.uniform(60, 100, n_samples),
    'Practice_Tests': np.random.randint(0, 20, n_samples)
}

# Create target variable with intentional relationships
data['Final_Score'] = (
    5 * data['Study_Hours'] +  # Strong positive correlation
    0.3 * data['Previous_Score'] +  # Moderate positive correlation
    2 * data['Sleep_Hours'] +  # Weak positive correlation
    0.2 * data['Attendance'] +  # Weak positive correlation
    1.5 * data['Practice_Tests'] +  # Moderate positive correlation
    np.random.normal(0, 5, n_samples)  # Random noise
)

df = pd.DataFrame(data)
print("Dataset Preview:")
print(df.head(10))
print("\nDataset Shape:", df.shape)
print("\nBasic Statistics:")
print(df.describe())

# ===========================
# 1. CORRELATION ANALYSIS
# ===========================

print("\n" + "="*80)
print("CORRELATION ANALYSIS")
print("="*80)

# Calculate correlation matrix
correlation_matrix = df.corr()
print("\nCorrelation Matrix:")
print(correlation_matrix.round(3))

# Correlation with target variable
target_correlation = correlation_matrix['Final_Score'].sort_values(ascending=False)
print("\nCorrelation with Final_Score (sorted):")
print(target_correlation)

# Statistical significance test for correlations
print("\n" + "="*80)
print("CORRELATION SIGNIFICANCE TEST (Pearson)")
print("="*80)

features = ['Study_Hours', 'Previous_Score', 'Sleep_Hours', 'Attendance', 'Practice_Tests']
for feature in features:
    corr, p_value = pearsonr(df[feature], df['Final_Score'])
    significance = "Significant" if p_value < 0.05 else "Not Significant"
    print(f"{feature:20s}: r = {corr:6.3f}, p-value = {p_value:.4f} ({significance})")

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, fmt='.2f', vmin=-1, vmax=1,
            cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Pairwise scatter plots for top correlated features
top_features = target_correlation[1:4].index.tolist()  # Top 3 features
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
fig.suptitle('Scatter Plots: Top 3 Correlated Features with Final Score', 
             fontsize=14, fontweight='bold')

for idx, feature in enumerate(top_features):
    axes[idx].scatter(df[feature], df['Final_Score'], alpha=0.6, s=30)
    axes[idx].set_xlabel(feature, fontweight='bold')
    axes[idx].set_ylabel('Final_Score', fontweight='bold')
    
    # Add regression line
    z = np.polyfit(df[feature], df['Final_Score'], 1)
    p = np.poly1d(z)
    axes[idx].plot(df[feature], p(df[feature]), "r--", linewidth=2, alpha=0.8)
    
    corr = correlation_matrix.loc[feature, 'Final_Score']
    axes[idx].set_title(f'Correlation: {corr:.3f}', fontweight='bold')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ===========================
# 2. FEATURE IMPORTANCE ANALYSIS
# ===========================

print("\n" + "="*80)
print("FEATURE IMPORTANCE ANALYSIS (Random Forest)")
print("="*80)

# Prepare data for Random Forest
X = df[features]
y = df['Final_Score']

# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=10)
rf_model.fit(X, y)

# Get feature importance scores
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nFeature Importance Scores:")
print(feature_importance)
print(f"\nModel R² Score: {rf_model.score(X, y):.4f}")

# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot for feature importance
axes[0].barh(feature_importance['Feature'], feature_importance['Importance'], 
             color='steelblue', alpha=0.8)
axes[0].set_xlabel('Importance Score', fontweight='bold')
axes[0].set_title('Feature Importance (Random Forest)', fontweight='bold', fontsize=12)
axes[0].grid(True, alpha=0.3, axis='x')

# Bar plot for correlation with target
target_corr_abs = target_correlation[1:].abs().sort_values(ascending=True)
axes[1].barh(target_corr_abs.index, target_corr_abs.values, 
             color='coral', alpha=0.8)
axes[1].set_xlabel('Absolute Correlation', fontweight='bold')
axes[1].set_title('Correlation with Final Score', fontweight='bold', fontsize=12)
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Combined comparison plot
comparison_df = pd.DataFrame({
    'Feature': features,
    'Importance': rf_model.feature_importances_,
    'Correlation': [abs(correlation_matrix.loc[f, 'Final_Score']) for f in features]
}).sort_values('Importance', ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(comparison_df))
width = 0.35

bars1 = ax.bar(x - width/2, comparison_df['Importance'], width, 
               label='Feature Importance', color='steelblue', alpha=0.8)
bars2 = ax.bar(x + width/2, comparison_df['Correlation'], width, 
               label='Abs Correlation', color='coral', alpha=0.8)

ax.set_xlabel('Features', fontweight='bold')
ax.set_ylabel('Score', fontweight='bold')
ax.set_title('Feature Importance vs Correlation Comparison', fontweight='bold', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(comparison_df['Feature'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("KEY INSIGHTS")
print("="*80)
print(f"Most important feature: {feature_importance.iloc[0]['Feature']}")
print(f"Most correlated feature: {target_correlation.index[1]}")
print(f"\nTop 3 features by importance: {', '.join(feature_importance['Feature'].head(3).tolist())}")

## Question 15: Multiple Subplots - Line, Bar, and Density Plots

**Concepts:**
- **Subplots**: Multiple plots arranged in a grid layout
- **Line plot**: Shows trends and patterns over time or continuous data
- **Bar plot**: Compares categorical data or discrete values
- **Density plot (KDE)**: Shows probability distribution of continuous variables
- **Matplotlib pyplot**: Interface for creating and arranging multiple plots
- **Figure and Axes**: Container objects for plots in matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Create a comprehensive dataset with multiple variables
np.random.seed(42)
n_points = 100

data = {
    'Month': pd.date_range(start='2023-01-01', periods=n_points, freq='D'),
    'Sales': np.random.normal(5000, 1000, n_points).cumsum() / 10,
    'Temperature': 20 + 10 * np.sin(np.linspace(0, 4*np.pi, n_points)) + np.random.normal(0, 2, n_points),
    'Customers': np.random.poisson(100, n_points),
    'Revenue': np.random.normal(15000, 3000, n_points),
    'Profit_Margin': np.random.normal(25, 5, n_points)
}

df = pd.DataFrame(data)

# Add categorical data for bar plots
categories = ['Electronics', 'Clothing', 'Food', 'Books', 'Sports']
category_data = {
    'Category': categories,
    'Sales_Volume': [45000, 38000, 42000, 28000, 35000],
    'Customer_Count': [1200, 1500, 1800, 900, 1100],
    'Avg_Rating': [4.5, 4.2, 4.7, 4.3, 4.4]
}
df_category = pd.DataFrame(category_data)

print("Time Series Data Preview:")
print(df.head())
print("\nCategorical Data Preview:")
print(df_category)

# ===========================
# COMPREHENSIVE SUBPLOT LAYOUT
# ===========================

# Create a 3x3 grid of subplots
fig = plt.figure(figsize=(18, 14))
fig.suptitle('Comprehensive Data Visualization: Line, Bar, and Density Plots', 
             fontsize=18, fontweight='bold', y=0.995)

# ===========================
# ROW 1: LINE PLOTS
# ===========================

# Line Plot 1: Sales Trend
ax1 = plt.subplot(3, 3, 1)
ax1.plot(df['Month'], df['Sales'], color='blue', linewidth=2, marker='o', 
         markersize=3, alpha=0.7)
ax1.set_title('Sales Trend Over Time', fontweight='bold', fontsize=11)
ax1.set_xlabel('Date', fontweight='bold')
ax1.set_ylabel('Sales ($)', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# Line Plot 2: Temperature Variation
ax2 = plt.subplot(3, 3, 2)
ax2.plot(df['Month'], df['Temperature'], color='red', linewidth=2, 
         marker='s', markersize=3, alpha=0.7)
ax2.set_title('Temperature Variation', fontweight='bold', fontsize=11)
ax2.set_xlabel('Date', fontweight='bold')
ax2.set_ylabel('Temperature (°C)', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.tick_params(axis='x', rotation=45)

# Line Plot 3: Multiple Variables
ax3 = plt.subplot(3, 3, 3)
ax3_twin = ax3.twinx()  # Create secondary y-axis
line1 = ax3.plot(df['Month'], df['Customers'], color='green', linewidth=2, 
                 label='Customers', marker='o', markersize=3, alpha=0.7)
line2 = ax3_twin.plot(df['Month'], df['Profit_Margin'], color='orange', linewidth=2, 
                      label='Profit Margin', marker='^', markersize=3, alpha=0.7)
ax3.set_title('Customers vs Profit Margin', fontweight='bold', fontsize=11)
ax3.set_xlabel('Date', fontweight='bold')
ax3.set_ylabel('Customers', fontweight='bold', color='green')
ax3_twin.set_ylabel('Profit Margin (%)', fontweight='bold', color='orange')
ax3.tick_params(axis='y', labelcolor='green')
ax3_twin.tick_params(axis='y', labelcolor='orange')
ax3.grid(True, alpha=0.3)
ax3.tick_params(axis='x', rotation=45)

# ===========================
# ROW 2: BAR PLOTS
# ===========================

# Bar Plot 1: Vertical Bar Chart
ax4 = plt.subplot(3, 3, 4)
bars1 = ax4.bar(df_category['Category'], df_category['Sales_Volume'], 
                color='steelblue', alpha=0.8, edgecolor='black', linewidth=1.2)
ax4.set_title('Sales Volume by Category', fontweight='bold', fontsize=11)
ax4.set_xlabel('Category', fontweight='bold')
ax4.set_ylabel('Sales Volume ($)', fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')
ax4.tick_params(axis='x', rotation=45)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{int(height):,}', ha='center', va='bottom', fontsize=8, fontweight='bold')

# Bar Plot 2: Horizontal Bar Chart
ax5 = plt.subplot(3, 3, 5)
bars2 = ax5.barh(df_category['Category'], df_category['Customer_Count'], 
                 color='coral', alpha=0.8, edgecolor='black', linewidth=1.2)
ax5.set_title('Customer Count by Category', fontweight='bold', fontsize=11)
ax5.set_xlabel('Customer Count', fontweight='bold')
ax5.set_ylabel('Category', fontweight='bold')
ax5.grid(True, alpha=0.3, axis='x')

# Add value labels
for bar in bars2:
    width = bar.get_width()
    ax5.text(width, bar.get_y() + bar.get_height()/2.,
             f'{int(width)}', ha='left', va='center', fontsize=8, fontweight='bold')

# Bar Plot 3: Grouped Bar Chart
ax6 = plt.subplot(3, 3, 6)
x_pos = np.arange(len(categories))
width = 0.35

# Normalize data for comparison
normalized_sales = df_category['Sales_Volume'] / 1000
normalized_customers = df_category['Customer_Count']

bars3 = ax6.bar(x_pos - width/2, normalized_sales, width, 
                label='Sales (×1000)', color='purple', alpha=0.8, edgecolor='black')
bars4 = ax6.bar(x_pos + width/2, normalized_customers, width, 
                label='Customers', color='teal', alpha=0.8, edgecolor='black')

ax6.set_title('Grouped Comparison: Sales vs Customers', fontweight='bold', fontsize=11)
ax6.set_xlabel('Category', fontweight='bold')
ax6.set_ylabel('Count', fontweight='bold')
ax6.set_xticks(x_pos)
ax6.set_xticklabels(categories, rotation=45)
ax6.legend(loc='upper right')
ax6.grid(True, alpha=0.3, axis='y')

# ===========================
# ROW 3: DENSITY PLOTS
# ===========================

# Density Plot 1: Single Variable
ax7 = plt.subplot(3, 3, 7)
df['Revenue'].plot(kind='density', ax=ax7, color='darkblue', linewidth=3, alpha=0.7)
ax7.fill_between(ax7.lines[0].get_xdata(), ax7.lines[0].get_ydata(), 
                 alpha=0.3, color='lightblue')
ax7.set_title('Revenue Distribution (KDE)', fontweight='bold', fontsize=11)
ax7.set_xlabel('Revenue ($)', fontweight='bold')
ax7.set_ylabel('Density', fontweight='bold')
ax7.grid(True, alpha=0.3)

# Add mean line
mean_revenue = df['Revenue'].mean()
ax7.axvline(mean_revenue, color='red', linestyle='--', linewidth=2, label=f'Mean: ${mean_revenue:.0f}')
ax7.legend()

# Density Plot 2: Comparison of Multiple Variables
ax8 = plt.subplot(3, 3, 8)
df['Customers'].plot(kind='density', ax=ax8, color='green', linewidth=2.5, 
                     alpha=0.7, label='Customers')
df['Temperature'].plot(kind='density', ax=ax8, color='red', linewidth=2.5, 
                       alpha=0.7, label='Temperature')
ax8.set_title('Multiple Variables Distribution', fontweight='bold', fontsize=11)
ax8.set_xlabel('Value', fontweight='bold')
ax8.set_ylabel('Density', fontweight='bold')
ax8.legend(loc='upper right')
ax8.grid(True, alpha=0.3)

# Density Plot 3: Histogram + Density Overlay
ax9 = plt.subplot(3, 3, 9)
ax9.hist(df['Profit_Margin'], bins=20, density=True, alpha=0.5, 
         color='gold', edgecolor='black', label='Histogram')
df['Profit_Margin'].plot(kind='density', ax=ax9, color='darkred', 
                         linewidth=3, label='KDE')
ax9.set_title('Profit Margin: Histogram + Density', fontweight='bold', fontsize=11)
ax9.set_xlabel('Profit Margin (%)', fontweight='bold')
ax9.set_ylabel('Density', fontweight='bold')
ax9.legend(loc='upper right')
ax9.grid(True, alpha=0.3)

# Add statistics text
stats_text = f"Mean: {df['Profit_Margin'].mean():.1f}%\nStd: {df['Profit_Margin'].std():.1f}%"
ax9.text(0.05, 0.95, stats_text, transform=ax9.transAxes, 
         fontsize=9, verticalalignment='top', bbox=dict(boxstyle='round', 
         facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

# ===========================
# ADDITIONAL: FOCUSED VIEW
# ===========================

# Create a more focused 1x3 layout showing one of each plot type
fig2, axes = plt.subplots(1, 3, figsize=(16, 5))
fig2.suptitle('Focused View: Line, Bar, and Density Plots', 
              fontsize=14, fontweight='bold')

# Line plot
axes[0].plot(df['Month'][:50], df['Sales'][:50], color='blue', 
             linewidth=2.5, marker='o', markersize=4)
axes[0].set_title('Line Plot: Sales Trend (First 50 Days)', fontweight='bold')
axes[0].set_xlabel('Date', fontweight='bold')
axes[0].set_ylabel('Sales ($)', fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Bar plot
bars = axes[1].bar(df_category['Category'], df_category['Avg_Rating'], 
                   color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'],
                   alpha=0.8, edgecolor='black', linewidth=1.5)
axes[1].set_title('Bar Plot: Average Rating by Category', fontweight='bold')
axes[1].set_xlabel('Category', fontweight='bold')
axes[1].set_ylabel('Average Rating', fontweight='bold')
axes[1].set_ylim([0, 5])
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].tick_params(axis='x', rotation=45)

# Add value labels
for bar in bars:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.1f}', ha='center', va='bottom', 
                 fontsize=10, fontweight='bold')

# Density plot
axes[2].hist(df['Temperature'], bins=25, density=True, alpha=0.4, 
             color='orange', edgecolor='black')
df['Temperature'].plot(kind='density', ax=axes[2], color='darkred', 
                       linewidth=3)
axes[2].set_title('Density Plot: Temperature Distribution', fontweight='bold')
axes[2].set_xlabel('Temperature (°C)', fontweight='bold')
axes[2].set_ylabel('Density', fontweight='bold')
axes[2].grid(True, alpha=0.3)

# Add mean and median lines
mean_temp = df['Temperature'].mean()
median_temp = df['Temperature'].median()
axes[2].axvline(mean_temp, color='blue', linestyle='--', linewidth=2, 
                label=f'Mean: {mean_temp:.1f}°C')
axes[2].axvline(median_temp, color='green', linestyle='--', linewidth=2, 
                label=f'Median: {median_temp:.1f}°C')
axes[2].legend()

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("PLOT SUMMARY")
print("="*80)
print("Generated 3×3 grid with:")
print("  - Row 1: 3 Line plots (trends, comparisons, multiple axes)")
print("  - Row 2: 3 Bar plots (vertical, horizontal, grouped)")
print("  - Row 3: 3 Density plots (single, multiple, histogram overlay)")
print("\nAdditional focused 1×3 layout created for clearer view.")

## Question 16: Multiple Linear Regression with Evaluation and Visualization

**Concepts:**
- **Multiple Linear Regression**: Predicts target using multiple independent variables
- **Equation**: y = β₀ + β₁x₁ + β₂x₂ + ... + βₙxₙ + ε
- **R² Score**: Proportion of variance explained by model (0 to 1, higher is better)
- **MSE (Mean Squared Error)**: Average squared difference between actual and predicted
- **RMSE (Root Mean Squared Error)**: Square root of MSE, in same units as target
- **Residuals**: Difference between actual and predicted values (errors)
- **Residual plot**: Should show random scatter if model is good
- **Train-test split**: Divide data to evaluate model on unseen data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler

# Create a comprehensive dataset for regression
np.random.seed(42)
n_samples = 300

# Generate features with realistic relationships
data = {
    'Study_Hours': np.random.uniform(1, 10, n_samples),
    'Previous_Score': np.random.uniform(50, 95, n_samples),
    'Sleep_Hours': np.random.uniform(4, 9, n_samples),
    'Attendance': np.random.uniform(60, 100, n_samples),
    'Practice_Tests': np.random.randint(0, 20, n_samples)
}

# Create target variable with known coefficients
# True equation: Score = 30 + 4*Study + 0.4*Previous + 2*Sleep + 0.1*Attendance + 1*Practice + noise
data['Final_Score'] = (
    30 +  # Intercept
    4.0 * data['Study_Hours'] +
    0.4 * data['Previous_Score'] +
    2.0 * data['Sleep_Hours'] +
    0.1 * data['Attendance'] +
    1.0 * data['Practice_Tests'] +
    np.random.normal(0, 3, n_samples)  # Random noise
)

df = pd.DataFrame(data)

print("Dataset Preview:")
print(df.head(10))
print("\nDataset Shape:", df.shape)
print("\nDataset Statistics:")
print(df.describe())

# Prepare features and target
feature_columns = ['Study_Hours', 'Previous_Score', 'Sleep_Hours', 'Attendance', 'Practice_Tests']
X = df[feature_columns]
y = df['Final_Score']

print("\n" + "="*80)
print("DATA PREPARATION")
print("="*80)
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures: {', '.join(feature_columns)}")
print(f"Target: Final_Score")

# Split data into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

# ===========================
# TRAIN THE MODEL
# ===========================

print("\n" + "="*80)
print("TRAINING MULTIPLE LINEAR REGRESSION MODEL")
print("="*80)

# Create and train the model
model = LinearRegression()
model.fit(X_train, y_train)

print("Model trained successfully!")

# Display model coefficients
print("\nModel Coefficients:")
print(f"Intercept (β₀): {model.intercept_:.4f}")
print("\nFeature Coefficients:")
for feature, coef in zip(feature_columns, model.coef_):
    print(f"  {feature:20s}: {coef:8.4f}")

# Create regression equation string
equation = f"Final_Score = {model.intercept_:.2f}"
for feature, coef in zip(feature_columns, model.coef_):
    equation += f" + {coef:.2f}×{feature}"
print(f"\nRegression Equation:\n{equation}")

# ===========================
# MAKE PREDICTIONS
# ===========================

# Predictions on training set
y_train_pred = model.predict(X_train)

# Predictions on test set
y_test_pred = model.predict(X_test)

# ===========================
# EVALUATE THE MODEL
# ===========================

print("\n" + "="*80)
print("MODEL EVALUATION METRICS")
print("="*80)

# Training set metrics
train_r2 = r2_score(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_mae = mean_absolute_error(y_train, y_train_pred)

print("\nTraining Set Performance:")
print(f"  R² Score:  {train_r2:.4f}")
print(f"  MSE:       {train_mse:.4f}")
print(f"  RMSE:      {train_rmse:.4f}")
print(f"  MAE:       {train_mae:.4f}")

# Testing set metrics
test_r2 = r2_score(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("\nTesting Set Performance:")
print(f"  R² Score:  {test_r2:.4f}")
print(f"  MSE:       {test_mse:.4f}")
print(f"  RMSE:      {test_rmse:.4f}")
print(f"  MAE:       {test_mae:.4f}")

# Model interpretation
print("\n" + "="*80)
print("MODEL INTERPRETATION")
print("="*80)
print(f"The model explains {test_r2*100:.2f}% of the variance in Final Score.")
print(f"On average, predictions are off by {test_mae:.2f} points.")
print(f"\nMost influential feature: {feature_columns[np.argmax(np.abs(model.coef_))]}")
print(f"Coefficient magnitude: {np.max(np.abs(model.coef_)):.4f}")

# ===========================
# RESIDUAL ANALYSIS
# ===========================

# Calculate residuals
train_residuals = y_train - y_train_pred
test_residuals = y_test - y_test_pred

print("\n" + "="*80)
print("RESIDUAL ANALYSIS")
print("="*80)
print(f"Training Residuals - Mean: {train_residuals.mean():.4f}, Std: {train_residuals.std():.4f}")
print(f"Testing Residuals - Mean: {test_residuals.mean():.4f}, Std: {test_residuals.std():.4f}")

# ===========================
# COMPREHENSIVE VISUALIZATION
# ===========================

fig = plt.figure(figsize=(18, 12))
fig.suptitle('Multiple Linear Regression: Comprehensive Analysis', 
             fontsize=18, fontweight='bold', y=0.995)

# Plot 1: Actual vs Predicted (Training Set)
ax1 = plt.subplot(2, 3, 1)
ax1.scatter(y_train, y_train_pred, alpha=0.5, s=30, color='blue', edgecolors='black', linewidth=0.5)
ax1.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 
         'r--', lw=2, label='Perfect Prediction')
ax1.set_xlabel('Actual Final Score', fontweight='bold')
ax1.set_ylabel('Predicted Final Score', fontweight='bold')
ax1.set_title(f'Training Set: Actual vs Predicted\nR² = {train_r2:.4f}', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Actual vs Predicted (Test Set)
ax2 = plt.subplot(2, 3, 2)
ax2.scatter(y_test, y_test_pred, alpha=0.6, s=40, color='green', edgecolors='black', linewidth=0.5)
ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
ax2.set_xlabel('Actual Final Score', fontweight='bold')
ax2.set_ylabel('Predicted Final Score', fontweight='bold')
ax2.set_title(f'Test Set: Actual vs Predicted\nR² = {test_r2:.4f}', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Coefficients Bar Plot
ax3 = plt.subplot(2, 3, 3)
colors = ['red' if c < 0 else 'green' for c in model.coef_]
bars = ax3.barh(feature_columns, model.coef_, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax3.axvline(x=0, color='black', linewidth=1)
ax3.set_xlabel('Coefficient Value', fontweight='bold')
ax3.set_title('Feature Coefficients', fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')

# Add value labels
for i, (feature, coef) in enumerate(zip(feature_columns, model.coef_)):
    ax3.text(coef, i, f' {coef:.3f}', va='center', fontweight='bold')

# Plot 4: Residual Plot (Training Set)
ax4 = plt.subplot(2, 3, 4)
ax4.scatter(y_train_pred, train_residuals, alpha=0.5, s=30, color='blue', edgecolors='black', linewidth=0.5)
ax4.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax4.set_xlabel('Predicted Final Score', fontweight='bold')
ax4.set_ylabel('Residuals', fontweight='bold')
ax4.set_title('Training Set: Residual Plot', fontweight='bold')
ax4.grid(True, alpha=0.3)

# Plot 5: Residual Plot (Test Set)
ax5 = plt.subplot(2, 3, 5)
ax5.scatter(y_test_pred, test_residuals, alpha=0.6, s=40, color='green', edgecolors='black', linewidth=0.5)
ax5.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax5.set_xlabel('Predicted Final Score', fontweight='bold')
ax5.set_ylabel('Residuals', fontweight='bold')
ax5.set_title('Test Set: Residual Plot', fontweight='bold')
ax5.grid(True, alpha=0.3)

# Plot 6: Residual Distribution
ax6 = plt.subplot(2, 3, 6)
ax6.hist(test_residuals, bins=30, density=True, alpha=0.6, color='orange', edgecolor='black')
# Add KDE plot
from scipy import stats
kde = stats.gaussian_kde(test_residuals)
x_range = np.linspace(test_residuals.min(), test_residuals.max(), 100)
ax6.plot(x_range, kde(x_range), 'r-', linewidth=2, label='KDE')
ax6.axvline(x=0, color='blue', linestyle='--', linewidth=2, label='Zero Error')
ax6.set_xlabel('Residual Value', fontweight='bold')
ax6.set_ylabel('Density', fontweight='bold')
ax6.set_title('Residual Distribution (Test Set)', fontweight='bold')
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ===========================
# ADDITIONAL: REGRESSION LINE PLOTS FOR INDIVIDUAL FEATURES
# ===========================

fig2, axes = plt.subplots(2, 3, figsize=(18, 10))
fig2.suptitle('Individual Feature Regression Lines (Single Variable Analysis)', 
              fontsize=16, fontweight='bold')

for idx, feature in enumerate(feature_columns):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]
    
    # Scatter plot
    ax.scatter(X_test[feature], y_test, alpha=0.5, s=30, 
               color='steelblue', edgecolors='black', linewidth=0.5)
    
    # Fit simple linear regression for visualization
    z = np.polyfit(X_test[feature], y_test, 1)
    p = np.poly1d(z)
    x_line = np.linspace(X_test[feature].min(), X_test[feature].max(), 100)
    ax.plot(x_line, p(x_line), "r-", linewidth=2.5, alpha=0.8, label='Regression Line')
    
    # Calculate correlation
    corr = np.corrcoef(X_test[feature], y_test)[0, 1]
    
    ax.set_xlabel(feature, fontweight='bold')
    ax.set_ylabel('Final Score', fontweight='bold')
    ax.set_title(f'{feature}\nCorr: {corr:.3f}, Coef: {model.coef_[idx]:.3f}', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

# Hide the extra subplot
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

# ===========================
# PREDICTION EXAMPLES
# ===========================

print("\n" + "="*80)
print("SAMPLE PREDICTIONS")
print("="*80)

# Show some example predictions
sample_indices = np.random.choice(X_test.index, 5, replace=False)
sample_predictions = pd.DataFrame({
    'Study_Hours': X_test.loc[sample_indices, 'Study_Hours'],
    'Previous_Score': X_test.loc[sample_indices, 'Previous_Score'],
    'Sleep_Hours': X_test.loc[sample_indices, 'Sleep_Hours'],
    'Attendance': X_test.loc[sample_indices, 'Attendance'],
    'Practice_Tests': X_test.loc[sample_indices, 'Practice_Tests'],
    'Actual_Score': y_test.loc[sample_indices],
    'Predicted_Score': model.predict(X_test.loc[sample_indices]),
    'Error': y_test.loc[sample_indices] - model.predict(X_test.loc[sample_indices])
})

print("\nSample Predictions from Test Set:")
print(sample_predictions.to_string(index=False))

print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)
print(f"Model Performance: R² = {test_r2:.4f}, RMSE = {test_rmse:.4f}")
print(f"The model {'performs well' if test_r2 > 0.8 else 'needs improvement'} on the test set.")
print(f"Residuals are {'normally distributed' if abs(test_residuals.mean()) < 0.5 else 'not well distributed'}.")